## Building A Chatbot
In this notebook We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation.

In [6]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
print(groq_api_key)

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


In [7]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x749e0127ac60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x749e0127bef0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is John and I am a Chief AI Engineer")])

AIMessage(content="Hi John,\n\nIt's nice to meet you! That's a fascinating role. What kind of AI projects are you working on these days?  \n\nI'm always eager to learn more about the advancements being made in the field.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 22, 'total_tokens': 75, 'completion_time': 0.096363636, 'prompt_time': 0.002137206, 'queue_time': 0.015209633, 'total_time': 0.098500842}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0a5b2824-657f-4d49-982d-780f2d8b0d89-0', usage_metadata={'input_tokens': 22, 'output_tokens': 53, 'total_tokens': 75})

In [9]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is John and I am a Chief AI Engineer"),
        AIMessage(content="Hello John! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are John, and you are a Chief AI Engineer.  \n\nIs there anything else you'd like me to remember about you, or any tasks you'd like my help with?  😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 97, 'total_tokens': 142, 'completion_time': 0.081818182, 'prompt_time': 0.004300593, 'queue_time': 0.020213386, 'total_time': 0.086118775}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-339b23c8-d947-4eec-8afe-c5850fd5e91f-0', usage_metadata={'input_tokens': 97, 'output_tokens': 45, 'total_tokens': 142})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [11]:
config={"configurable":{"session_id":"chat1"}}

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is John and I am a Chief AI Engineer")],
    config=config
)

In [13]:
response.content

"Hello John, it's a pleasure to meet you! \n\nBeing a Chief AI Engineer is a fascinating role. What kind of projects are you working on these days? \n\nI'm always interested in learning more about the applications of AI and how people are using it to solve real-world problems.\n"

In [14]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is John.  😊  \n\nI remember that from our introduction.  How can I help you today?  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 102, 'total_tokens': 130, 'completion_time': 0.050909091, 'prompt_time': 0.005154532, 'queue_time': 0.017283546, 'total_time': 0.056063623}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-85194604-2eb2-4035-ad75-21ad292d507e-0', usage_metadata={'input_tokens': 102, 'output_tokens': 28, 'total_tokens': 130})

In [15]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and I don't know your name. If you'd like to tell me your name, I'd be happy to know! 😊\n"

In [16]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is Eric")],
    config=config1
)
response.content

"Hi Eric! It's nice to meet you.  What can I do for you today?  😄 \n"

In [17]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is Eric!  I remember now. 😊  \n\nIs there anything else I can help you with, Eric? \n'